Hadoop Programming
====


```
Usage: hadoop [--config confdir] [COMMAND] [GENERIC_OPTIONS] [COMMAND_OPTIONS]
```

 * **Commands** 
 
|Command | Description|
|:--|:--|
|archive|create a Hadoop archive|
|jar | to run a jar file|
|classpath
|distcp||
|fs||
|version||


 * **Generic options:**
 
|Generic Option | Description |
|:---|:---|
|`-conf <config file>`|Specify the config file|
|`-D <propert=value>`|Set the value of a property|
|`-fs <local> or <namenode:port>```| specify a namenode|
|`-jt <local> or <jobtracker:port>` | |
|`-files <comma separated list of files>`|Specify  files to be copied to the map reduce cluster|
|`-libjars <comma seperated list of jars>`|Specify jar files to include in the classpath|
|`-archives <comma separated list of archives>`||

## Writing a Mapper Class

* **Context** objects are used to write the output of map() function

## Hadoop Data Types for Keys & Values

* **WritableComparable** can be used for both keys & values
* **Writable** interface is for efficiently serializing objects for input and output. can be used for values

|Class|Description|
|:---:|:----:|
|BooleanWritable|Standard boolean writable|
|ByteWritable|a single byte|
|DoubleWritable|a double|
|FloatWritable|a float|
|IntWritable|an integer|
|LongWritable|a long|
|Text|to store text using UTF8 format|
|NullWritable|Placeholder when the key or value is not needed|

## Minimial Hadoop Program

The most simple Hadoop program one can write must contain the minimial components of a Hadoop program. Minimial Hadoop program uses the default mapper, i.e. Mapper.class, and the default reducer Reducer.class

 * Mapper.class reads record line by line. The key is the offset from begining of line (LongWritable) and value is Text
 * The default reducer (Reducer.class ) writes its input directly to output as is, without any aggregation
 
**miminal_hadoop.java**
```java
import java.io.IOException;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.MRJobConfig;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;

public class MinimalProgram {
    public static void main(String[] args) throws Exception {
        Configuration conf = new Configuration();
        Job job = Job.getInstance(conf, "minimal program");
        job.setJarByClass(minimal.class);
        FileInputFormat.addInputPath(job, new Path(args[0]));
        FileOutputFormat.setOutputPath(job, new Path(args[1]));
        System.exit(job.waitForCompletion(true) ? 0 : 1);
    }
}

```

 * Compile

```bash
hduser@ubuntu$ javac -cp [provide class path] minimal.java
```

 * Archive into a jar file
 
```bash
hduser@ubuntu$ jar cvf minimal.jar *.class
```

 * Run Hadoop

```bash
hduser@ubuntu$ hadoop jar minimal.jar MinimalProgram /example/input.txt /example/out
```

 * Retrieve the results from HDFS
 
```bash
hduser@ubuntu$ hadoop dfs -cat /example/out/part-r-00000
0	Master Kenobi, you disappoint me.
34	Yoda holds you in such high esteem.
70	Surely you can do better!
```

#### Understanding components of this minimal Hadoop program

 * **Libraries**
 * **Configurations**
 * **Job setup**
 


## Hadoop Program Configuraions

 * **Default properties** mapred-default.xml
 * **Specifying properties in a configuration directory** 
 
 ```hadoop --config <config_dir> jar <jarfile> <class_name>```
 
 * **Specify the configurations explicitly within the program**
 
 ```java
 Configuration conf = new Configuration();
 conf.set("property1", "value1");
 ```
 * Modify the configuration properties with generic options 
 
 ```
 hadoop jar <jarfile> <class_name> -D property=value
 ```

## Wordcount


**wordcount_v1.java**
```java
import java.io.IOException;
import org.apache.hadoop.conf.*;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;
import org.apache.hadoop.util.Tool;
import org.apache.hadoop.util.ToolRunner;


import org.apache.hadoop.mapreduce.lib.map.TokenCounterMapper;
import org.apache.hadoop.mapreduce.lib.reduce.IntSumReducer;

import org.apache.hadoop.io.Text;
import org.apache.hadoop.io.IntWritable;

public class wordcount extends Configured implements Tool {

  public int run(String[] args) throws Exception {
    Job job = new Job(getConf());
    job.setJarByClass(getClass());
    job.setMapperClass(TokenCounterMapper.class);
    job.setReducerClass(IntSumReducer.class);

    job.setOutputKeyClass(Text.class);
    job.setOutputValueClass(IntWritable.class);

    FileInputFormat.addInputPath(job, new Path(args[0]));
    FileOutputFormat.setOutputPath(job, new Path(args[1]));
    return job.waitForCompletion(true) ? 0 : 1;
  }

  public static void main(String [] args) throws Exception {
    int exitCode = ToolRunner.run(new wordcount(), args);
    System.exit(exitCode);
  }

}
```


**Built-in Mapper and Reducers:**

  * TokenCounterMapper  
  ```java
  import org.apache.hadoop.mapreduce.lib.map.TokenCounterMapper;
  ```
  
  * IntSumReducer 
  ```java
  import org.apache.hadoop.mapreduce.lib.reduce.IntSumReducer;
  ```
  
  * Set the types of output keys and values of reducer
  ```java
    job.setOutputKeyClass(Text.class);
    job.setOutputValueClass(IntWritable.class);
  ```
  
Put everything together
  
```java
import java.io.IOException;
import org.apache.hadoop.conf.*;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;
import org.apache.hadoop.util.Tool;
import org.apache.hadoop.util.ToolRunner;

import org.apache.hadoop.mapreduce.lib.map.TokenCounterMapper;
import org.apache.hadoop.mapreduce.lib.reduce.IntSumReducer;

import org.apache.hadoop.io.Text;
import org.apache.hadoop.io.IntWritable;

public class wordcount extends Configured implements Tool {

  public int run(String[] args) throws Exception {
    Job job = new Job(getConf());
    job.setJarByClass(getClass());
    job.setMapperClass(TokenCounterMapper.class);
    job.setReducerClass(IntSumReducer.class);

    job.setOutputKeyClass(Text.class);
    job.setOutputValueClass(IntWritable.class);

    FileInputFormat.addInputPath(job, new Path(args[0]));
    FileOutputFormat.setOutputPath(job, new Path(args[1]));
    return job.waitForCompletion(true) ? 0 : 1;
  }

  public static void main(String [] args) throws Exception {
    int exitCode = ToolRunner.run(new wordcount(), args);
    System.exit(exitCode);
  }

}
  ```